# 👩‍🏫 *MEGAN: Multi Explanation Graph Attention Network*

The `Megan` model is a *graph neural network (GNN)* architecture designed to be *self-explainable* - meaning that it outputs not only the primary target property prediction but also an explanation of its own decision simultaneously. The model achieves this through a sophisticated multi-channel attention mechanism that identifies the most relevant nodes and edges in the input graph which provide evidence for or against the predicted property.

## Why Multi-Channel Explanations?

Unlike traditional GNNs that produce a single prediction, MEGAN employs multiple "explanation channels" - parallel processing pathways that each learn to focus on different aspects of the graph structure. For regression tasks, typically two channels are used: one identifying graph features that contribute positively to the prediction, and another identifying features that contribute negatively. This dual-channel approach provides more nuanced and interpretable explanations than single-attention mechanisms.

## Self-Supervised Explanation Training

The key innovation in MEGAN is the *explanation co-training* approach. During training, the model not only learns to make accurate predictions but also learns to make those predictions explainable. This is achieved through a self-supervised explanation loss that forces each channel's attention weights to be predictive of the target property by themselves - without relying on the actual node features. This constraint ensures that the attention mechanisms capture genuinely important structural patterns rather than arbitrary correlations.

This notebook demonstrates the complete workflow: dataset preparation, model configuration, training with explanation co-training, and generating interpretable predictions with visual explanations.

# 🧪 Loading the Dataset

As a first step, we can load the dataset that will be used to demonstrate the model capabilities. This example dataset contains roughtly 10k molecules which are annotated with the Crippen's logP values as target property. This value can be deterministically calculated by `rdkit`.

In [ ]:
import os
import csv
import pandas as pd

# --- loading the dataset ---

# This is the path to the folder in which this notebook is located in.
PATH: str = os.getcwd()
# For the purpose of this example, we use the dataset file "clogp.csv" that is located 
# in the same "examples" folder as this notebook.
DATASET_PATH: str = os.path.join(PATH, "clogp.csv")

# The example dataset is a CSV file consisting of molecular data in the format of 
# SMILES strings along with their corresponding clogP annotations. We can use 
# pandas, for example, to load this dataset.
dataset: pd.DataFrame = pd.read_csv(DATASET_PATH)

print('\n dataset size:')
print(len(dataset))
print('\n dataset preview:')
dataset.head()

# ⚙️ Dataset Processing

After loading the dataset in the previous step, the individual elements are still represented as SMILES strings. The `Megan` model, however, is a *graph neural network (GNN)* and thus requires a *graph representation* of these molecule. For this purpose we can use the `MoleculeProcessing` class from the `visual_graph_datasets` library, which converts the SMILES strings into appropriate graph representations.

More specifically, the `MoleculeProcessing` class provides the `process(smiles: str) -> dict` method which converts the SMILES string into a dictionary representation of the graph. This dictionary contains various numpy arrayes representing different aspects of the graph such as list of the atom and bond types, the connectivity of the graph in the form of an edge index list and the already featurized node and edge feature arrays.

In [ ]:
from rich.pretty import pprint
from visual_graph_datasets.processing.molecules import MoleculeProcessing

# The SMILES representation of caffeine, which we want to use as an example for 
# the conversion into a graph representation.
SMILES = 'C1=CC=C2C=C(CCN)C=CC2=C1'

processing = MoleculeProcessing()
graph: dict = processing.process(SMILES)

print('\n attributes of the graph dictionary:')
pprint(list(graph.keys()))
print('\n graph dictionary:')
pprint(graph)

In addition to the conversion of a SMILES to a graph dictionary representation, the `MoleculeProcessing` class also provides the `visualize_as_figure(smiles: str) -> plt.Figure, ndarray` method which can be used to visualize the molecule as a 2D figure (using RDKit under the hood).

Crucially, this method returns a tuple of two values, the first being the matplotlib figure object and the second being an array of `node_positions` containing the 2D coordinates of each atom of the molecule in the coordinate system of the figure. This information is essential later on for the visualization of the node importance explanations provided by the `Megan` model.

In [ ]:
fig, node_positions = processing.visualize_as_figure(SMILES, width=1000, height=1000)

# To illustrate the the node positions, we will plot the node index of each atom on top 
# of the molecular illustration.
for node_idx, (x, y) in enumerate(node_positions):
    ax = fig.axes[0]
    ax.text(x, 1000-y, str(node_idx), fontsize=10, color='black')
    
fig.show()


# 🤖 Setting up the Model

In the next step, we need to initialize the `Megan` model. This can be done by instantiating the `Megan` class. The constructor accpets a larger number of parameters, the most important ones for the core functionality of the model being briefly explained in the code cell below:

In [ ]:
from graph_attention_student import Megan

model = Megan(
    # --- prediction config ---
    # These first parameters need to specify how many node and edge features are used 
    # for the graphs that will be input to the model. This information can be obtained 
    # from the MoleculeProcessing object that was used to create the graphs.
    node_dim=processing.get_num_node_attributes(),
    edge_dim=processing.get_num_edge_attributes(),
    # This list determines the structure of the message passing layers in the encoder.
    # In this case we use three message passing steps with 64 hidden units each.
    units=[64, 64, 64], 
    # Likewise, this list determines the structure of the final prediction MLP. For 
    # this one, it is important that the last entry matches the number of target 
    # values to be predicted (i.e. 1 for regression, or the number of classes
    # for classification).
    final_units=[64, 32, 1],
    # This flag indicates that we want to perform a regression task. For binary 
    # classification this would be 'bce' and for multi-class classification
    # 'classification'.
    prediction_mode='regression',
    # The learning rate controls how large the steps are that are taken during
    # gradient descent optimization.
    learning_rate=1e-4,
    # --- explanation config ---
    # Following parameters determine the configuration of the explanation mechanism.
    importance_mode='regression',
    # Most importantly setting this importance factor to a non-zero value activates 
    # the explanation consistency co-training loss.
    importance_factor=1.0,
    # Likewise, this factor modulates the weight of the sparsity loss that is 
    # applied to the explanation masks during training.
    sparsity_factor=0.5,
    # This value also influences the sparsity of the resulting explanations. The higher 
    # this value, the more sparse the explanation will be (meaning that less nodes 
    # will be highlighted as important). Lower values will highlight more nodes.
    importance_offset=1.0,
)

print('\n model summary:')
model

# 🤖 Model Training

The model is trained with `pytorch_lightning`, which provides a high-level interface for training PyTorch models. 

To load the training data, the `graph_attention_student` package provides the special `SmilesDataset` class which is a subclass of `torch.utils.data.IterableDataset` that provides an efficient lazy-loaded datastream from the given CSV file. The dataset class implements reservoir sampling to ensure proper shuffling of the data during training. This dataset class will then act as an iterator instance for PyG `Data` objects representing the individual molecular graphs in the dataset and can then be used to create a PyG `DataLoader` instance.

Finally, the model can be trained by instantiating a `pytorch_lightning.Trainer` object and calling its `fit` method with the model and the training data loader as arguments.

In [ ]:
import pytorch_lightning as pl
from torch_geometric.loader import DataLoader
from graph_attention_student import SmilesDataset

# --- training preparation ---

dataset = SmilesDataset(
    dataset=DATASET_PATH,
    smiles_column='smiles',
    target_columns=['value'],
    processing=processing,
    # This will randomize the order of the streamed dataset - effectively 
    # acting as a shuffling mechanism.
    reservoir_sampling=True,
)

loader_train = DataLoader(
    dataset, 
    batch_size=64, 
    #shuffle=True,
    drop_last=True,
    num_workers=4,
    prefetch_factor=2,
)

trainer = pl.Trainer(
    max_epochs=150,
    accelerator='auto',
    devices='auto',
)

In [ ]:
# --- model training ---
# This function will start the actual training of the model and may therefore take
# a few minutes.
trainer.fit(
    model,
    train_dataloaders=loader_train,
)

# Important: After the training is completed, we need to explicitly put the model 
# into evaluation mode to ensure that the BatchNorm layers work propertly during 
# inference.
model.eval()

# 💾 Saving and Loading the Model

After training a model, one may want to save that model to the disk to continue training later or to use it for inference. The `Megan` class provides convenient methods for saving and loading the model. The model can be saved to a checkpoint file using the `save(path: str)` method and can be loaded back using the class method `load(path: str) -> Megan`.

In [ ]:
MODEL_PATH = os.path.join(PATH, "model.ckpt")

# --- model saving ---
# The model can be easily saved into a persistent file representation using the 
# the "save" method.
model.save(MODEL_PATH)

# --- model loading ---
# Equally, an object instance of the model can be recreated from the saved file 
# representation using the "load" method of the Megan class.
model: Megan = Megan.load(MODEL_PATH)

# 🔮 Model Prediction

To query a prediction for an unkown molecule, the `Megan` model provides the `forward_graph(graph: dict) -> dict` method which accepts a graph dictionary representation of the molecule (as provided by the `MoleculeProcessing` class) and returns a dictionary containing the predicted property value as well as various other artifacts of the model forward pass.

In [ ]:
from rdkit import Chem

graph = processing.process(SMILES)

results: dict = model.forward_graph(graph)
print('\n prediction results:')
pprint(list(results.keys()))

print('\n predicted value:')
pprint(results['graph_output'].item())

print('\n true value:')
true_value = Chem.Crippen.MolLogP(Chem.MolFromSmiles(SMILES))
pprint(true_value)


The previous cell shows how to obtain the main property prediction from the trained model. However, one main use case of the `Megan` model is the explaination of its own predictions. The function `megan_prediction_report` provides a convenient interface to visualize not only the prediction but the explanation as well. This function will create a report PDF that includes the visualization of the molecule, the predicted property value and a heatmap visualization of the node and edge importance scores as provided by the model's attention mechanism. To do this, the function needs to receive the SMILES string to be explained, the trained model as well as the `MoleculeProcessing` instance that was used to featurize the dataset. The report will be saved to the specified `output_path`.

In [ ]:
from IPython.display import IFrame
from graph_attention_student.torch.advanced import megan_prediction_report

output_path = os.path.join(PATH, "megan_prediction_report.pdf")
megan_prediction_report(
    value=SMILES,
    model=model,
    processing=processing,
    output_path=output_path,
)

IFrame(output_path, width=600, height=600)